In [1]:
%load_ext autoreload

# Overlaps, Norms and Expectation values

In [2]:
# file: mps/expectation.py

import numpy as np
import mps.state

## Scalar products

The computation of scalar products is the algorithm that underlies many other problems, such as approximating states, computing expectation values or minimizing energies. For instance, an expectation value $\langle \psi |O|\psi\rangle$ is tantamount to a scalar product between $|\psi\rangle$ and the vector $O|\psi\rangle.$ The distance between states, used in MPS simplification algorithms, is also a collection of scalar products $\|\psi-\phi\|^2 = \langle\psi|\psi\rangle + \langle\phi|\phi\rangle -2 \mathrm{Re}\langle\psi|\phi\rangle$, etc.

The MPS have a simple structure where the scalar product becomes a simple graph that can be contracted with a cost $\mathcal{O}(N)$ that is linear in the size of the state.

<img src="figures/scalar-product.svg" style="max-width:50%; width: 400px">

The computation proceeds by grouping tensors into a tensor that can be contracted with subsequent layers of tensors. For instance, in the example above we would start by grouping $A$ and $A^\star$ into a single tensor $T_A$. Doing the same with the other tensors transforms the scalar product into a contraction of matrices

$$| \phi \rangle = \sum \text{Tr} A^{i_1} C^{i_2} E^{i_3} G^{i_4} | i_1, i_2, i_3, i_4 \rangle$$

$$| \psi \rangle = \sum \text{Tr} B^{i_1} D^{i_2} F^{i_3} H^{i_4} | i_1, i_2, i_3, i_4 \rangle$$

$$\langle\psi|\phi\rangle = \mathrm{Tr}(O_1 O_2 O_3 O_4)$$

Graphically, this would be

<img src="figures/scalar-product-phase1.svg" style="max-width:50%; width:400px">

## Environments

When we contract the $O_i$ tensors above to construct the scalar product, what we are doing is studying the overlap between the basis of states that we use to build our states $|\psi\rangle$ and $|\phi\rangle$ with 1, 2, 3 or four components.

For instance, $(O_1)_{\beta,\alpha}$ is the overlap between
$$|\psi_1(\alpha)\rangle = \sum_{i_1} A^{i_1}_{1,\alpha}|i_1\rangle$$
and the state
$$|\phi_1(\beta)\rangle = \sum_{i_1} B^{i_1}_{1,\beta}|i_1\rangle,$$
so that
$$(O_1)_{\beta,\alpha} = \langle \phi_1(\beta)|\psi_1(\alpha)\rangle.$$

Similarly, we can construct a new environment $(\rho_2)_{\delta,\gamma}$ given by
$$\rho_2 = O_1 O_2$$
or the scalar productthat
$$(\rho_2)_{\gamma,\delta} = \langle \phi_2(\delta)|\psi_2(\gamma)\rangle$$
between the basis states
$$|\psi_2(\gamma)\rangle = \sum_{i_1,i_2,\alpha} A^{i_1}_{1,\alpha} C^{i_1}_{\alpha,\gamma}|i_1,i_2\rangle$$
$$|\phi_2(\delta)\rangle = \sum_{i_1,i_2,\alpha} B^{i_1}_{1,\beta} D^{i_2}_{\beta,\delta}|i_1,i_2\rangle.$$

We will now implement two sets of functions that implement these environments, both from the left and from the right of the MPS.

### Left-to-right environments

The following function takes tensors from the "ket" and the "bra" and extends a previously built environment. When we start creating environments, $\rho_0 = []$ is the empty tensor. The sequence of contractions is shown below and is optimal in the number of operations.

<img src="figures/left-to-right-contraction.svg" style="max-width:90%; width: 600px">

In [3]:
# file: mps/expectation.py

def begin_environment():
    """Initiate the computation of a left environment from two MPS."""
    return np.ones((1,1), dtype=np.float64)

def close_environment(ρ):
    """Extract the scalar product from the last environment."""
    return ρ[0,0]

def update_left_environment(B, A, rho, operator=None):
    """Extend the left environment with two new tensors, 'B' and 'A' coming
    from the bra and ket of a scalar product. If an operator is provided, it
    is contracted with the ket."""
    if operator is not None:
        A = np.einsum("ji,aib->ajb", operator, A)
    rho = np.einsum("li,ijk->ljk", rho, A)
    return np.einsum("lmn,lmk->nk", B.conj(), rho)

### Right-to-left environment

We have a similar procedure for right environments. These are used in various algorithms.

<img src="figures/right-to-left-contraction.svg" style="max-width:90%; width: 600px">

In [4]:
# file: mps/expectation.py

def udpate_right_environment(B, A, rho, operator=None):
    """Extend the left environment with two new tensors, 'B' and 'A' coming
    from the bra and ket of a scalar product. If an operator is provided, it
    is contracted with the ket."""
    if operator is not None:
        A = np.einsum("ji,aib->ajb", operator, A)
    rho = np.einsum("ijk,kn->ijn", A, rho)
    return np.einsum("imn,lmn->il", rho, B)

## Applications

### Scalar product

Using these functions, we can implement a scalar product. We initiate the left environment and then loop over sites, adding sites one at a time. 

In [5]:
# file: mps/expectation.py

def scprod(ϕ, ψ):
    """Compute the scalar product between matrix product states <ϕ|ψ>."""
    rho = begin_environment()
    for i in range(ψ.size):
        rho = update_left_environment(ϕ[i], ψ[i], rho)
    return close_environment(rho)

The scalar product is used to define the norm of an MPS, $\|\psi\|^2=\langle\psi|\psi\rangle.$

### Single-site expectation value

The expectation value of an operator acting on a single site is obtained by constructing environments and only changing one of the kets to include the contraction with the observable.

In [6]:
# file: mps/expectation.py

def expectation1_non_canonical(ψ, O, site):
    """Compute the expectation value <ψ|O|ψ> of an operator O acting on 'site'"""
    ρL = begin_environment()
    for i in range(0, ψ.size):
        A = ψ[i]
        if i == site:
            OL = update_left_environment(A, A, ρL, operator=O)
        elif i > site:
            OL = update_left_environment(A, A, OL)
        ρL = update_left_environment(A, A, ρL)
    return close_environment(OL)/close_environment(ρL)

### Multiple single-site expectation values

Quite often we need to compute multiple expectation values, one acting on each site. This is the case, for instance, when we want to study the density of particles on a lattice, given by $n_i =\langle a^\dagger_i a_i\rangle,$ or the polarization of a quantum ferromagnet.

The following function implements this, by computing
$$\{\langle \psi | O_i |\psi\rangle\}_{i=1}^{N},$$
where the $i$-th operator acts on the $i$-th site. The function takes an argument that may be a list of operators $[O_1,O_2,\ldots,O_N],$ or just one operator that is repeated over all sites. The function has more than 50% reduction in time costs with respect to calling `expectation1_non_nanonical()` again and again, due to reusing precomputed left-environments.

In [7]:
# file: mps/expectation.py

def get_operator(O, i):
    #
    # This utility function is used to guess the operator acting on site 'i'
    # If O is a list, it corresponds to the 'i'-th element. Otherwise, we
    # use operator 'O' everywhere.
    #
    if type(O) == list:
        return O[i]
    else:
        return O

In [8]:
# file: mps/expectation.py

def all_expectation1_non_canonical(ψ, O, tol=0):
    """Return all expectation values of operator O acting on ψ. If O is a list
    of operators, a different one is used for each site."""
    
    Oenv = []
    ρL = begin_environment()
    for i in range(ψ.size):
        A = ψ[i]
        Oenv = [update_left_environment(A, A, ρO) for ρO in Oenv] + \
               [update_left_environment(A, A, ρ, operator=get_operator(O,i))]
        ρL = update_left_environment(A, A, ρ)
    return np.array(map(end_environment, Oenv))/end_environment(ρL)

### Expectation of a product of operators

The following function computes the expectation value of a product of operators, combining all the methods above:
$$\langle \psi | O_1 \otimes O_2 \otimes \cdots \otimes O_N |\psi\rangle$$
We pass the operators in a list $[O_1, O_2,\ldots, O_N].$

In [9]:
# file: mps/expectation.py

def product_expectation(ψ, operator_list):
    rho = begin_environment(ρ)
    
    for i in range(ψ.size):
        rho = update_left_environment(ψ[i].conj(), ψ[i], rho, operator = operator_list[i])
    
    return close_environment(ρ)

-----

# Tests

In [14]:
# file: mps/test/test_expectation.py

import unittest
import mps.state
from mps.expectation import *

def bit2state(b):
    if b:
        return [0,1]
    else:
        return [1,0]

class TestExpectation(unittest.TestCase):
    
    def test_scprod_basis(self):
        #
        # Test that scprod() can be used to project onto basis states
        for nbits in range(1,8):
            # We create a random MPS
            ψmps = mps.state.random(2, nbits, 2)
            ψwave = ψmps.tovector()
            
            # We then create the basis of all states with well defined
            # values of the qubits
            conf = np.arange(0, 2**nbits, dtype=np.uint8)
            conf = np.reshape(conf, (2**nbits, 1))
            conf = np.unpackbits(conf, axis=1)
            
            # Finally, we loop over the basis states, verifying that the
            # scalar product is the projection onto the state
            for (n, bits) in enumerate(conf):
                proj = ψwave[n]
                ϕmps = mps.state.product(map(bit2state, bits[-nbits:]))
                self.assertEqual(proj, scprod(ϕmps, ψmps))

    def test_norm_random(self):
        #
        # Test the norm on our sample states
        for nbits in range(1, 8):
            self.assertAlmostEqual(mps.state.GHZ(nbits).norm2(),
                                   1.0, places=10)
            self.assertAlmostEqual(mps.state.W(nbits).norm2(),
                                   1.0, places=10)
            self.assertAlmostEqual(mps.state.graph(nbits).norm2(),
                                   1.0, places=10)
            self.assertAlmostEqual(mps.state.AKLT(nbits).norm2(),
                                   1.0, places=10)
        
        # Test that the norm works on random states
        for nbits in range(1,8):
            for _ in range(10):
                # We create a random MPS
                ψmps = mps.state.random(2, nbits, 2)
                ψwave = ψmps.tovector()
                self.assertAlmostEqual(ψmps.norm2(), np.vdot(ψwave,ψwave))
    
    def test_expected1_density(self):
        def random_wavefunction(n):
            ψ = np.random.rand(n) - 0.5
            return ψ / np.linalg.norm(ψ)
        
        O = np.array([[0,0],[0,1]])
        
        for nbits in range(1,8):
            ψGHZ = mps.state.GHZ(nbits)
            ψW = mps.state.W(nbits)
            for i in range(nbits):
                self.assertAlmostEqual(ψGHZ.expectation1(O,i), 0.5)
                self.assertAlmostEqual(ψW.expectation1(O,i), 1/nbits)

        #
        # When we create a spin wave, 'O' detects the density of the
        # wave with the right magnitude
        for nbits in range(2,8):
            for _ in range(10):
                # We create a random MPS
                ψwave = random_wavefunction(nbits)
                ψmps = mps.state.wavepacket(ψwave)
                for i in range(nbits):
                    si = expectation1_non_canonical(ψmps, O, i)
                    self.assertAlmostEqual(si, ψwave[i]**2)
                    xi = ψmps.expectation1(O, i)
                    self.assertEqual(si, xi)

In [15]:
%autoreload
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestExpectation'])
unittest.TextTestRunner(verbosity=2).run(suite1);

test_expected1_density (__main__.TestExpectation) ... ok
test_norm_random (__main__.TestExpectation) ... ok
test_scprod_basis (__main__.TestExpectation) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.117s

OK
